<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import the libraries**

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

# **Upload Train file**

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
#file_in = open('/content/drive/My Drive/Colab Notebooks/data/Train_20K.txt')
#df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.txt', sep='\t')
df_train.head()
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.txt', sep='\t')
df_test.head()

,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [0]:
from google.colab import files
uploaded = files.upload()
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()


In [8]:
from google.colab import files
uploaded = files.upload()
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

Saving Test_5K.txt to Test_5K (1).txt


,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [38]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<br.*?>|\d+')
  train_data_X = [regex.sub('', x) for x in train_data_X]
  return train_data_X

In [0]:

def tokenize(train_data_X):
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(y) for y in x] for x in train_data_X]
  return train_data_X

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = [len(word_to_ix), 1]
          else:
              val = word_to_ix[word][1]
              val += 1
              word_to_ix[word][1] = val
              
  #word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [0]:
def reduce_vocab(word_to_ix):
  new_word_to_ix = {}
  for word in word_to_ix:
    val = word_to_ix[word][1]
    if val >= 5:
      new_word_to_ix[word] = len(new_word_to_ix)
  new_word_to_ix['UNKNOWN'] = len(new_word_to_ix)
  return new_word_to_ix

In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

In [57]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = remove_stopwords(train_data_X)
train_data_X = lemmatize(train_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [58]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = remove_stopwords(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
word_to_ix = generate_word_ids(train_data_X + test_data_X)
word_to_ix = reduce_vocab(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
HIDDEN_LAYER_NEURONS = 100
print(VOCAB_SIZE, word_to_ix)

24672 {'john': 0, 'water': 1, 'given': 2, 'u': 3, 'genuinely': 4, 'enjoyable': 5, 'film': 6, 'certainly': 7, 'without': 8, 'shocking': 9, 'esque': 10, 'moment': 11, 'tamer': 12, 'older': 13, 'stuff': 14, 'pink': 15, 'flamingo': 16, 'pecker': 17, 'back': 18, 'early': 19, 'mainstream': 20, 'stage': 21, 'reminds': 22, 'viewer': 23, 'kind': 24, 'humor': 25, 'evident': 26, 'polyester': 27, 'overall': 28, 'really': 29, 'fun': 30, 'comedy': 31, 'great': 32, 'first': 33, 'two': 34, 'season': 35, 'series': 36, 'strange': 37, 'funny': 38, 'drama': 39, 'element': 40, 'bill': 41, 'mother': 42, 'struggling': 43, 'usual': 44, 'problem': 45, 'life': 46, 'bit': 47, 'depressing': 48, 'mix': 49, 'well': 50, 'th': 51, 'probably': 52, 'dropped': 53, 'soon': 54, 'became': 55, 'one': 56, 'funniest': 57, 'bbc': 58, 'ever': 59, 'made': 60, 'chemistry': 61, 'ben': 62, 'character': 63, 'always': 64, 'many': 65, 'brilliant': 66, 'memorable': 67, 'sketch': 68, 'christmas': 69, 'special': 70, 'hilarious': 71, 'rea

In [60]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

In [61]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons)
    self.lin2 = nn.Linear(hidden_layer_neurons, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

In [0]:
bow = BOWClassifier(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
from torch.autograd import Variable
import time

bow_vec = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec.append(Variable(ins_bow_vector).cuda())
  
label = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label.append(Variable(label_vector).cuda())



In [67]:
# the training loop
for epoch in range(20):
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        bow.zero_grad()
        probs = bow(bow_vec[ind]) # forward pass
        loss = loss_function(probs, label[ind])
        loss.backward()
        opt.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 17.91825556755066
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.06686806678772
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.630160093307495
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.54193615913391
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 17.727216005325317
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.023131847381592
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.175440549850464
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.125416040420532
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 17.724934101104736
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.233875036239624
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.305370807647705
Epoch - 10 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 18.33890986442566
Epoch - 11 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.055214881896973
Epoch - 12

In [54]:
import torch
torch.save(bow,'model.json')

from google.colab import files
model = files.download("model.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
import pickle

pickle_out = open("word_to_ix.pickle","wb")
pickle.dump(word_to_ix, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix.pickle")


In [57]:
from google.colab import files
temp_test = files.upload()


Saving model.json to model (1).json


In [0]:
import torch
bow = torch.load(io.BytesIO(temp_test['model.json']))

In [59]:
from google.colab import files
temp_test = files.upload()

Saving word_to_ix.pickle to word_to_ix (1).pickle


In [0]:
pickle_in = open("word_to_ix.pickle","rb")
word_to_ix = pickle.load(pickle_in)

In [68]:
print('--- AFTER TRAINING ---')
count = 0
for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = bow(bow_vec)
    pred = np.argmax(logprobs.cpu().data.numpy())
    label = test_data_Y[ind]
    if label == ix_to_label[pred]:
      count += 1
      
print(float(count)/ len(test_data_X))

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


0.8436
